In [1]:
import gurobipy as gp
import pandas as pd
from gurobipy import GRB

In [18]:
plants = ["Afyon", "Konya", "Import (Izmir)"]
br_exist = ["Istanbul", "Ankara"]
br_pot = ["Izmir", "Sakarya", "Adana"]
dc = ["Istanbul", "Izmir", "Antalya", "Bursa", "Kayseri", "Export (Izmir)"]  # Distribution Centers

build_cost = {'Istanbul': 0, 'Ankara': 0, 'Izmir': 75, 'Sakarya': 70, 'Adana': 68}
expand_cost = {'Istanbul': 0, 'Ankara': 0, 'Izmir': 30, 'Sakarya': 27, 'Adana': 25}

plant_br_data = {
    "Malt Plants": ["Afyon", "Konya", "Import (Izmir)"],
    "Istanbul": [0.026, 0.037, 0.032],
    "Ankara": [0.017, 0.017, 0.033],
    "Izmir": [0.020, 0.031, 0.004],
    "Sakarya": [0.019, 0.030, 0.028],
    "Adana": [0.032, 0.022, 0.048]
}

ship_plant_br = pd.DataFrame(plant_br_data)
ship_plant_br.set_index("Malt Plants", inplace=True)

# Data from the table
br_dc_data = {
    "Breweries": [
        "Istanbul", "Ankara", "Izmir", "Sakarya", "Adana"
    ],
    "Istanbul": [0.000, 0.032, 0.040, 0.011, 0.067],
    "Izmir": [0.040, 0.041, 0.000, 0.034, 0.064],
    "Antalya": [0.052, 0.039, 0.032, 0.041, 0.040],
    "Bursa": [0.017, 0.027, 0.023, 0.011, 0.060],
    "Kayseri": [0.055, 0.023, 0.062, 0.045, 0.024],
    "Export (Izmir)": [0.042, 0.043, 0.002, 0.036, 0.066]
}

# Create DataFrame
ship_br_dc = pd.DataFrame(br_dc_data)
ship_br_dc.set_index("Breweries", inplace=True)

# Data from the table
demand_data = {
    "Dist. Centers": [
        "Istanbul", "Izmir", "Antalya", "Bursa", "Kayseri", "Export (Izmir)", "Total"
    ],
    1: [103, 74, 50, 60, 102, 13, 402],
    2: [110, 80, 53, 75, 110, 13, 441],
    3: [125, 90, 60, 85, 125, 15, 500]
}

# Create DataFrame
demand = pd.DataFrame(demand_data)
demand.set_index("Dist. Centers", inplace=True)

malt_cap = {"Afyon": 30, "Konya": 68, "Import (Izmir)": 20}

V = {"Afyon": 8.333, "Konya": 8.333, "Import (Izmir)": 9.091}



br_cap_exist ={"Istanbul": 220, "Ankara": 200}

br_cap_new = 70
br_cap_expand = 50


periods = [1, 2, 3]

['Istanbul', 'Ankara', 'Izmir', 'Sakarya', 'Adana']

In [13]:
# VARIABLES

m = gp.Model("Single Period Distribution")

x = m.addVars(plants, br_exist + br_pot, periods, lb=0.0, name='x')  # plant i --> brewery k
y = m.addVars(br_exist + br_pot, dc, periods, lb=0.0, name='y') # brewery j --> dc k

b = m.addVars(br_exist + br_pot, periods, vtype=GRB.BINARY, name='b') # brewery at site l
e = m.addVars(br_exist + br_pot, periods, vtype=GRB.BINARY, name='e') # expansion at site l


m.update()

In [ ]:
# CONSTRAINTS

m.addConstrs((b[i, t] == 1 for i in br_exist for t in periods), name='Existing Breweries')
m.addConstrs((e[i, t] == 0 for i in br_exist for t in periods), name='No Expansion at Existing Breweries')

build_continuity = m.addConstrs((b[l, t-1] <= b[l, t] for l in br_pot for t in periods[1:]), name='Build Continuity')
expansion_continuity = m.addConstrs((e[l, t-1] <= e[l, t] for l in br_pot for t in periods[1:]), name='Expansion Continuity')
build_expand = m.addConstrs((e[l, t] <= b[l, t] for l in br_pot for t in periods), name='Build-Expand')

build_constraint = m.addConstrs((x[i, j, t] <= malt_cap[i] * b[j, t] for i in plants for j in br_exist + br_pot for t in periods), name='No shipping to breweries not build')


dc_demand = m.addConstrs((gp.quicksum(y[j, k, t] for j in br_exist + br_pot) >= demand.loc[k, t] for k in dc for t in periods), name='Demand')

malt_capacity = m.addConstrs((gp.quicksum(x[i, j, t] for j in br_exist) <= malt_cap[i] for i in plants for t in periods), name='Malt Capacity')

brew_capacity_ex = m.addConstrs((gp.quicksum(x[i, j, t] * V[i] for i in plants) <= br_cap_exist[j] for j in br_exist for t in periods), name='Existing Brewery Capacity')

brew_capacity_pot = m.addConstrs((gp.quicksum(x[i, j, t] * V[i] for i in plants) <= br_cap_new * b[j, t] + br_cap_expand * e[j, t] for j in br_pot for t in periods), name='Brewery Capacity at New Sites')


transshipment = m.addConstrs((gp.quicksum(x[i, j, t] * V[i] for i in plants) >= gp.quicksum(y[j, k, t] for k in dc) for j in br_exist for t in periods), name='Transshipment')

m.update()